# Analyse de Sentiment des News Hybrides

Ce notebook récupère les news collectées via le système hybride et applique l'analyse de sentiment avec le modèle FinBERT fine-tuné.

**Workflow:**
1. Chargement des news depuis `hybrid_news_mapped.csv`
2. Initialisation du modèle FinBERT de sentiment
3. Analyse de sentiment pour chaque article
4. Enrichissement des données avec les scores de sentiment
5. Export des résultats

## 1. Import des bibliothèques

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✓ Bibliothèques importées avec succès")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ Device disponible: {'GPU' if torch.cuda.is_available() else 'CPU'}")

d:\Documents\ECE\2025\PFE\FinsightAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Bibliothèques importées avec succès
✓ PyTorch version: 2.5.1+cu121
✓ Device disponible: GPU


## 2. Chargement des données de news

In [2]:
# Chemins des fichiers
NEWS_CSV_PATH = Path("../Pipeline_Recup_Donnees/data/raw/news/hybrid_news_mapped.csv")

# Chargement du fichier CSV
print(f"📂 Chargement des news depuis: {NEWS_CSV_PATH}")
df_news = pd.read_csv(NEWS_CSV_PATH)

# Affichage des informations de base
print(f"\n✓ {len(df_news)} lignes chargées")
print(f"✓ Colonnes: {list(df_news.columns)}")
print(f"\n📊 Dimensions: {df_news.shape}")
print(f"📰 News uniques: {df_news['url'].nunique()}")
print(f"📈 Actifs concernés: {df_news['asset'].nunique()}")

# Affichage d'un échantillon
print("\n📋 Aperçu des données:")
df_news.head()

📂 Chargement des news depuis: ..\Pipeline_Recup_Donnees\data\raw\news\hybrid_news_mapped.csv

✓ 4280 lignes chargées
✓ Colonnes: ['date', 'title', 'url', 'source', 'language', 'event_type', 'event_category', 'base_impact_score', 'affects', 'asset', 'relevance_score', 'matched_events']

📊 Dimensions: (4280, 12)
📰 News uniques: 377
📈 Actifs concernés: 18

📋 Aperçu des données:


,date,title,url,source,language,event_type,event_category,base_impact_score,affects,asset,relevance_score,matched_events
0,2026-01-09 06:15:00,Mercosur - kauppasopimus ratkaisevassa nestyks...,https://yle.fi/a/74-20203286,yle.fi,Finnish,trade_policy,macro,8,NaN,TESLA,19.2,"technology, automotive"
1,2026-01-09 06:15:00,Mercosur - kauppasopimus ratkaisevassa nestyks...,https://yle.fi/a/74-20203286,yle.fi,Finnish,trade_policy,macro,8,NaN,APPLE,9.6,technology
2,2026-01-09 06:15:00,Mercosur - kauppasopimus ratkaisevassa nestyks...,https://yle.fi/a/74-20203286,yle.fi,Finnish,trade_policy,macro,8,NaN,AMAZON,9.6,technology
3,2026-01-09 06:15:00,Mercosur - kauppasopimus ratkaisevassa nestyks...,https://yle.fi/a/74-20203286,yle.fi,Finnish,trade_policy,macro,8,NaN,STELLANTIS,9.6,automotive
4,2026-01-09 06:15:00,Mercosur - kauppasopimus ratkaisevassa nestyks...,https://yle.fi/a/74-20203286,yle.fi,Finnish,trade_policy,macro,8,NaN,CASIC,9.6,technology


In [3]:
# Vérification des données manquantes dans les colonnes clés
print("🔍 Vérification des valeurs manquantes:")
missing_data = df_news[['title', 'url', 'asset', 'event_type']].isnull().sum()
print(missing_data)

# Dédupliquer par URL pour avoir des news uniques
print(f"\n📰 Avant déduplication: {len(df_news)} entrées")
df_news_unique = df_news.drop_duplicates(subset=['url']).copy()
print(f"📰 Après déduplication: {len(df_news_unique)} news uniques")

# Afficher quelques exemples de titres
print("\n📌 Exemples de titres de news:")
for idx, title in enumerate(df_news_unique['title'].head(5), 1):
    print(f"  {idx}. {title[:80]}...")

🔍 Vérification des valeurs manquantes:
title         0
url           0
asset         0
event_type    0
dtype: int64

📰 Avant déduplication: 4280 entrées
📰 Après déduplication: 377 news uniques

📌 Exemples de titres de news:
  1. Mercosur - kauppasopimus ratkaisevassa nestyksessä 25 vuoden neuvottelujen jälke...
  2. Iran in 2026 : Isolated in a Turbulent Ocean...
  3. Rüstung im Würgegriff : Warum Lockheed und Boeing ohne Antimon am Boden bleiben ...
  4. Tariff shock could weigh on jobs while easing inflation , Fed Research suggests...
  5. Aktien von Advantest , Renesas und Softbank brechen ein : Politischer Schock aus...


## 3. Initialisation du modèle FinBERT

In [10]:
# Configuration du device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  Device: {device}")

# Chemin du modèle
MODEL_PATH = "./news_finbert_sentiment_model"

# Chargement du modèle et du tokenizer
print(f"\n📥 Chargement du modèle depuis: {MODEL_PATH}")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=2,
    use_safetensors=True
)
model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

print("✓ Modèle FinBERT chargé et prêt")
print(f"✓ Configuration: {model.config.num_labels} labels (0=Negative, 1=Positive)")

🖥️  Device: cuda

📥 Chargement du modèle depuis: ./news_finbert_sentiment_model


OSError: Error no file named model.safetensors found in directory ./news_finbert_sentiment_model.

In [ ]:
# Test rapide du modèle
test_text = "Stock market reaches new all-time high as economic growth surges"
print(f"\n🧪 Test du modèle avec: '{test_text}'")

# Tokenization de test
test_encoding = tokenizer(
    test_text,
    add_special_tokens=True,
    max_length=512,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

# Prédiction de test
with torch.no_grad():
    test_outputs = model(
        input_ids=test_encoding['input_ids'].to(device),
        attention_mask=test_encoding['attention_mask'].to(device)
    )
    test_probs = F.softmax(test_outputs.logits, dim=1)[0]
    test_prediction = torch.argmax(test_outputs.logits, dim=1).item()

print(f"✓ Prédiction: {'Positive' if test_prediction == 1 else 'Negative'}")
print(f"✓ Prob Negative: {test_probs[0].item():.2%}")
print(f"✓ Prob Positive: {test_probs[1].item():.2%}")

## 4. Fonction d'analyse de sentiment

In [ ]:
def analyze_sentiment(text, model, tokenizer, device, max_length=512):
    """
    Analyse le sentiment d'un texte avec le modèle FinBERT
    
    Args:
        text: Texte à analyser (titre de la news)
        model: Modèle FinBERT
        tokenizer: Tokenizer
        device: CPU ou GPU
        max_length: Longueur maximale (512 pour FinBERT)
    
    Returns:
        dict avec sentiment, confiance, et probabilités
    """
    
    # Gestion des textes vides ou null
    if not text or pd.isna(text) or len(str(text).strip()) == 0:
        return {
            'sentiment': 'Unknown',
            'confidence': 0.0,
            'prob_negative': 0.5,
            'prob_positive': 0.5
        }
    
    # Tokenization
    encoding = tokenizer(
        str(text),
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Prédiction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=1)[0]
        
        prediction = torch.argmax(logits, dim=1).item()
        confidence = probabilities[prediction].item()
    
    # Labels: 0=Negative, 1=Positive
    sentiment_label = "Positive" if prediction == 1 else "Negative"
    
    return {
        'sentiment': sentiment_label,
        'confidence': confidence,
        'prob_negative': probabilities[0].item(),
        'prob_positive': probabilities[1].item()
    }

print("✓ Fonction analyze_sentiment définie")

## 5. Analyse de sentiment sur toutes les news

In [ ]:
print("="*80)
print("🤖 DÉBUT DE L'ANALYSE DE SENTIMENT")
print("="*80)

# Préparer les listes pour stocker les résultats
sentiments = []
confidences = []
prob_negatives = []
prob_positives = []

total_news = len(df_news_unique)
print(f"\n📰 Analyse de {total_news} news uniques...\n")

# Analyser chaque news
for idx, row in df_news_unique.iterrows():
    # Utiliser le titre pour l'analyse
    title = row['title']
    
    # Analyser le sentiment
    result = analyze_sentiment(title, model, tokenizer, device)
    
    # Stocker les résultats
    sentiments.append(result['sentiment'])
    confidences.append(result['confidence'])
    prob_negatives.append(result['prob_negative'])
    prob_positives.append(result['prob_positive'])
    
    # Afficher la progression tous les 10%
    progress = (len(sentiments) / total_news) * 100
    if len(sentiments) % max(1, total_news // 10) == 0:
        print(f"   ⏳ Progression: {progress:.0f}% ({len(sentiments)}/{total_news})")

# Ajouter les colonnes au DataFrame
df_news_unique['sentiment'] = sentiments
df_news_unique['confidence'] = confidences
df_news_unique['prob_negative'] = prob_negatives
df_news_unique['prob_positive'] = prob_positives

print(f"\n✓ Analyse terminée pour {total_news} news")
print("="*80)

In [ ]:
# Afficher un aperçu des résultats
print("\n📊 Aperçu des résultats:")
print(df_news_unique[['title', 'sentiment', 'confidence', 'asset']].head(10))

## 6. Statistiques et analyse des résultats

In [ ]:
print("="*80)
print("📈 STATISTIQUES GLOBALES")
print("="*80)

# Statistiques de sentiment
sentiment_counts = df_news_unique['sentiment'].value_counts()
print(f"\n🎭 Distribution des sentiments:")
for sentiment, count in sentiment_counts.items():
    percentage = (count / len(df_news_unique)) * 100
    print(f"  {sentiment:10s}: {count:4d} news ({percentage:.1f}%)")

# Statistiques de confiance
print(f"\n🎯 Confiance du modèle:")
print(f"  Moyenne     : {df_news_unique['confidence'].mean():.2%}")
print(f"  Médiane     : {df_news_unique['confidence'].median():.2%}")
print(f"  Min         : {df_news_unique['confidence'].min():.2%}")
print(f"  Max         : {df_news_unique['confidence'].max():.2%}")

# Distribution par niveau de confiance
high_conf = (df_news_unique['confidence'] > 0.8).sum()
medium_conf = ((df_news_unique['confidence'] > 0.6) & (df_news_unique['confidence'] <= 0.8)).sum()
low_conf = (df_news_unique['confidence'] <= 0.6).sum()

print(f"\n📊 Distribution par confiance:")
print(f"  Haute (>80%)    : {high_conf:4d} news ({(high_conf/len(df_news_unique)*100):.1f}%)")
print(f"  Moyenne (60-80%): {medium_conf:4d} news ({(medium_conf/len(df_news_unique)*100):.1f}%)")
print(f"  Faible (<60%)   : {low_conf:4d} news ({(low_conf/len(df_news_unique)*100):.1f}%)")

print("="*80)

In [ ]:
# Analyse par type d'événement
print("\n📰 Sentiment par type d'événement:")
event_sentiment = df_news_unique.groupby(['event_type', 'sentiment']).size().unstack(fill_value=0)
print(event_sentiment)

# Sentiment moyen par actif
print("\n📈 Top 10 actifs par nombre de news:")
top_assets = df_news_unique['asset'].value_counts().head(10)
for rank, (asset, count) in enumerate(top_assets.items(), 1):
    asset_data = df_news_unique[df_news_unique['asset'] == asset]
    positive_pct = (asset_data['sentiment'] == 'Positive').sum() / len(asset_data) * 100
    avg_conf = asset_data['confidence'].mean()
    print(f"  {rank:2d}. {asset:15s}: {count:3d} news | {positive_pct:.1f}% positive | Conf: {avg_conf:.2%}")

In [ ]:
# Exemples de news avec sentiment positif et confiance élevée
print("\n✅ Top 5 news POSITIVES (haute confiance):")
positive_high = df_news_unique[
    (df_news_unique['sentiment'] == 'Positive') & 
    (df_news_unique['confidence'] > 0.8)
].nlargest(5, 'confidence')

for idx, row in positive_high.iterrows():
    print(f"\n  📌 {row['title'][:70]}...")
    print(f"     Asset: {row['asset']} | Confiance: {row['confidence']:.2%}")

# Exemples de news avec sentiment négatif et confiance élevée
print("\n❌ Top 5 news NÉGATIVES (haute confiance):")
negative_high = df_news_unique[
    (df_news_unique['sentiment'] == 'Negative') & 
    (df_news_unique['confidence'] > 0.8)
].nlargest(5, 'confidence')

for idx, row in negative_high.iterrows():
    print(f"\n  📌 {row['title'][:70]}...")
    print(f"     Asset: {row['asset']} | Confiance: {row['confidence']:.2%}")

## 7. Réintégration avec toutes les lignes du dataset original

In [ ]:
# Merger les résultats de sentiment avec le dataset complet (avec duplicatas)
# Cela permet d'avoir le sentiment pour chaque association news-actif

print("🔄 Fusion des résultats avec le dataset complet...")

# Sélectionner les colonnes de sentiment
sentiment_cols = ['url', 'sentiment', 'confidence', 'prob_negative', 'prob_positive']
df_sentiment = df_news_unique[sentiment_cols].copy()

# Fusionner avec le dataset original
df_news_enriched = df_news.merge(df_sentiment, on='url', how='left')

print(f"✓ Dataset enrichi: {len(df_news_enriched)} lignes")
print(f"✓ Colonnes ajoutées: sentiment, confidence, prob_negative, prob_positive")

# Vérifier les valeurs manquantes
missing_sentiment = df_news_enriched['sentiment'].isna().sum()
if missing_sentiment > 0:
    print(f"⚠️  {missing_sentiment} lignes sans sentiment (probablement des URLs manquantes)")

# Afficher un aperçu
print("\n📋 Aperçu du dataset enrichi:")
df_news_enriched[['date', 'title', 'asset', 'sentiment', 'confidence', 'relevance_score']].head()

## 8. Export des résultats

In [ ]:
# Générer un timestamp pour les fichiers de sortie
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Chemin de sortie
output_dir = Path("../Pipeline_Recup_Donnees/data/raw/news")
output_dir.mkdir(parents=True, exist_ok=True)

# Export CSV complet (avec tous les duplicatas actif-news)
output_csv_full = output_dir / f"hybrid_news_sentiment_full_{timestamp}.csv"
df_news_enriched.to_csv(output_csv_full, index=False)
print(f"✓ Dataset complet sauvegardé: {output_csv_full}")
print(f"  ({len(df_news_enriched)} lignes)")

# Export CSV news uniques uniquement
output_csv_unique = output_dir / f"hybrid_news_sentiment_unique_{timestamp}.csv"
df_news_unique.to_csv(output_csv_unique, index=False)
print(f"\n✓ News uniques sauvegardées: {output_csv_unique}")
print(f"  ({len(df_news_unique)} news uniques)")

# Export JSON pour intégration API
output_json = output_dir / f"hybrid_news_sentiment_{timestamp}.json"
df_news_unique.to_json(output_json, orient='records', indent=2)
print(f"\n✓ Format JSON sauvegardé: {output_json}")

In [ ]:
# Créer un rapport récapitulatif
summary_report = {
    "analysis_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    "total_news": len(df_news_unique),
    "total_associations": len(df_news_enriched),
    "sentiment_distribution": {
        "positive": int(sentiment_counts.get('Positive', 0)),
        "negative": int(sentiment_counts.get('Negative', 0)),
        "unknown": int(sentiment_counts.get('Unknown', 0))
    },
    "confidence_stats": {
        "mean": float(df_news_unique['confidence'].mean()),
        "median": float(df_news_unique['confidence'].median()),
        "min": float(df_news_unique['confidence'].min()),
        "max": float(df_news_unique['confidence'].max())
    },
    "top_assets": df_news_enriched['asset'].value_counts().head(10).to_dict(),
    "event_types": df_news_enriched['event_type'].value_counts().to_dict()
}

# Export du rapport en JSON
output_summary = output_dir / f"sentiment_analysis_summary_{timestamp}.json"
import json
with open(output_summary, 'w', encoding='utf-8') as f:
    json.dump(summary_report, f, indent=2, ensure_ascii=False)

print(f"\n✓ Rapport récapitulatif sauvegardé: {output_summary}")

print("\n" + "="*80)
print("✅ ANALYSE DE SENTIMENT TERMINÉE")
print("="*80)
print("\n📁 Fichiers générés:")
print(f"  1. {output_csv_full.name}")
print(f"  2. {output_csv_unique.name}")
print(f"  3. {output_json.name}")
print(f"  4. {output_summary.name}")

## 9. Visualisation des résultats (Optionnel)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration du style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 10)

# Créer une figure avec plusieurs sous-graphiques
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Distribution des sentiments
sentiment_counts.plot(kind='bar', ax=axes[0, 0], color=['green', 'red', 'gray'])
axes[0, 0].set_title('Distribution des Sentiments', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Sentiment')
axes[0, 0].set_ylabel('Nombre de News')
axes[0, 0].tick_params(axis='x', rotation=0)

# 2. Distribution de la confiance
axes[0, 1].hist(df_news_unique['confidence'], bins=30, color='skyblue', edgecolor='black')
axes[0, 1].set_title('Distribution de la Confiance', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Confiance')
axes[0, 1].set_ylabel('Fréquence')
axes[0, 1].axvline(df_news_unique['confidence'].mean(), color='red', linestyle='--', label='Moyenne')
axes[0, 1].legend()

# 3. Top 10 actifs
top_10_assets = df_news_enriched['asset'].value_counts().head(10)
top_10_assets.plot(kind='barh', ax=axes[1, 0], color='coral')
axes[1, 0].set_title('Top 10 Actifs les Plus Mentionnés', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Nombre de News')
axes[1, 0].set_ylabel('Actif')

# 4. Sentiment par type d'événement
event_sentiment_pct = df_news_unique.groupby('event_type')['sentiment'].value_counts(normalize=True).unstack()
event_sentiment_pct.plot(kind='bar', stacked=True, ax=axes[1, 1], color=['green', 'red', 'gray'])
axes[1, 1].set_title('Sentiment par Type d\'Événement', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Type d\'Événement')
axes[1, 1].set_ylabel('Proportion')
axes[1, 1].legend(title='Sentiment')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(output_dir / f"sentiment_analysis_viz_{timestamp}.png", dpi=300, bbox_inches='tight')
print(f"\n✓ Visualisations sauvegardées: sentiment_analysis_viz_{timestamp}.png")
plt.show()